In [ ]:
# --- 1. CONFIGURATION ---
import os
import numpy as np
import sys
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adamax

sys.path.append('..')
from data.loader import load_ecg_images
from models.cnn_model import build_base_cnn

DATA_PATH = "C:/Users/votre_nom/Desktop/ECG_Dataset"

# Charger tous les dossiers une fois pour toutes
normal = load_ecg_images(os.path.join(DATA_PATH, "Normal"))
abnormal = load_ecg_images(os.path.join(DATA_PATH, "Abnormal"))
history_mi = load_ecg_images(os.path.join(DATA_PATH, "History_MI"))
current_mi = load_ecg_images(os.path.join(DATA_PATH, "Current_MI"))

# ==========================================
# STAGE 1 : BINARY (Normal vs Pathological)
# ==========================================
X_bin = np.concatenate([normal, abnormal, history_mi, current_mi], axis=0)[..., np.newaxis]
y_bin = np.concatenate([np.zeros(len(normal)), np.ones(len(abnormal)+len(history_mi)+len(current_mi))], axis=0)

X_train, X_temp, y_train, y_temp = train_test_split(X_bin, y_bin, test_size=0.3, stratify=y_bin, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

model = build_base_cnn(num_classes=2)
model.compile(optimizer=Adamax(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, to_categorical(y_train, 2), validation_data=(X_val, to_categorical(y_val, 2)), epochs=30)
model.save("stage1_binary.h5")

# ==========================================
# STAGE 2 : 3-CLASS (Normal, Current MI, Others)
# ==========================================
# On regroupe Abnormal et History_MI dans "Others"
X_3cl = np.concatenate([normal, current_mi, abnormal, history_mi], axis=0)[..., np.newaxis]
y_3cl = np.concatenate([np.zeros(len(normal)), np.ones(len(current_mi)), np.full(len(abnormal)+len(history_mi), 2)], axis=0)

X_train, X_temp, y_train, y_temp = train_test_split(X_3cl, y_3cl, test_size=0.3, stratify=y_3cl, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# TRANSFER LEARNING
old_model = load_model("stage1_binary.h5")
old_model.pop() # Enlever la couche de sortie (2)
new_model = Sequential(old_model.layers)
new_model.add(Dense(3, activation='softmax')) # Ajouter nouvelle couche (3)

new_model.compile(optimizer=Adamax(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, to_categorical(y_train, 3), validation_data=(X_val, to_categorical(y_val, 3)), epochs=30)
new_model.save("stage2_3class.h5")

# ==========================================
# STAGE 3 : FINAL 4-CLASS
# ==========================================
y_4cl = np.concatenate([np.zeros(len(normal)), np.ones(len(current_mi)), np.full(len(abnormal), 2), np.full(len(history_mi), 3)], axis=0)
X_train, X_temp, y_train, y_temp = train_test_split(X_3cl, y_4cl, test_size=0.3, stratify=y_4cl, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# TRANSFER LEARNING FINAL
old_model = load_model("stage2_3class.h5")
old_model.pop()
final_model = Sequential(old_model.layers)
final_model.add(Dense(4, activation='softmax'))

final_model.compile(optimizer=Adamax(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
final_model.fit(X_train, to_categorical(y_train, 4), validation_data=(X_val, to_categorical(y_val, 4)), epochs=30)

# --- RÉSULTATS FINAUX ---
acc = final_model.evaluate(X_test, to_categorical(y_test, 4))[1]
print(f"✅ Précision finale de la méthode Cascade : {acc*100:.2f}%")